## **Supervised ML regression algorithm to predict next round team value (CT & T)**
## **Algorithm**

Input data: Obtained DataFrames from the previous prediction:
- ct_predicted_value
- t_predicted_value

Preprocces the data to create a new traget column, next round team value (**nxt_rnd_ct_val** & **nxt_rnd_t_val**)

In [1]:
import pandas as pd

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.metrics import mean_squared_error, r2_score

from sklearn.linear_model import Lasso

from lightgbm import LGBMRegressor
from joblib import dump, load

In [2]:
pd.set_option('display.max_columns', 30)
pd.set_option('display.max_rows', 30)

### Load Data

In [3]:
ct_df = pd.read_csv('../data/processed/3_base_predict_next_rnd_ct_val.csv')
t_df = pd.read_csv('../data/processed/3_base_predict_next_rnd_t_val.csv')

In [4]:
display(ct_df.head())
display(t_df.head())

,Unnamed: 0,file,round,wp_ct_val,nade_ct_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,ct_val_real,ct_val_pred,nxt_rnd_ct_val
0,0,0,1,1000.0,550,5,5,0.5,0.5,0,0,4550,4078.134589,18450
1,1,0,2,10100.0,1100,4,0,1.0,0.0,1,0,18450,17819.702711,9550
2,2,0,3,4125.0,900,0,1,0.0,0.0,0,1,9550,7038.468589,1600
3,3,0,4,1000.0,0,0,3,0.0,1.0,0,2,1600,1452.468928,23350
4,4,0,5,15500.0,1400,0,4,0.0,1.0,0,3,23350,22676.205763,26400


,Unnamed: 0,file,round,wp_t_val,nade_t_val,ct_alive,t_alive,ct_winner,bomb_planted,ct_cons_wins,t_cons_wins,t_val_real,t_val_pred,nxt_rnd_t_val
0,0,0,1,1166.666667,1200,5,5,0.5,0.5,0,0,3850,3943.272665,5300
1,1,0,2,3687.500000,50,4,0,1.0,0.0,1,0,5300,6290.616771,22900
2,2,0,3,11700.000000,2450,0,1,0.0,0.0,0,1,22900,19600.790638,19650
3,3,0,4,11700.000000,1600,0,3,0.0,1.0,0,2,19650,22568.098741,21750
4,4,0,5,12750.000000,1700,0,4,0.0,1.0,0,3,21750,24459.855175,19900


### Define Features and Target

In [5]:
CT_FEATS = ['file', 'round', 'wp_ct_val', 'nade_ct_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 'ct_val_pred']
T_FEATS = ['file', 'round', 'wp_t_val', 'nade_t_val', 'ct_alive', 't_alive', 'ct_winner', 'bomb_planted', 'ct_cons_wins', 't_cons_wins', 't_val_pred']

CT_TARGET = 'nxt_rnd_ct_val'
T_TARGET = 'nxt_rnd_t_val'

### Preprocessor

In [6]:
#Preprocessor

numeric_transformer = Pipeline(steps=[('scaler', StandardScaler())])

preprocessor_ct = ColumnTransformer(transformers=[('num', numeric_transformer, CT_FEATS)])
preprocessor_t = ColumnTransformer(transformers=[('num', numeric_transformer, T_FEATS)])

### **SPLITS**

In [7]:
ct_train, ct_test = train_test_split(ct_df)
t_train, t_test = train_test_split(t_df)

In [8]:
print(ct_train.shape, ct_test.shape)
print(t_train.shape, t_test.shape)

(233109, 14) (77704, 14)
(233109, 14) (77704, 14)


### **MODEL**

#### We take **LightGBM** as the regressor algorithm, like the previous prediction

In [9]:
regressor = LGBMRegressor(boosting_type='gbdt', 
                       bagging_freq=1, 
                       bagging_fraction = 0.9, 
                       n_estimators=100)

# regressor = Lasso()

In [10]:
# ct_model = Pipeline(steps=[('ct_preprocessor', preprocessor_ct),
#                            ('regressor', regressor)])

t_model = Pipeline(steps=[('t_preprocessor', preprocessor_t),
                           ('regressor', regressor)])

In [11]:
# ct_model.fit(ct_train[CT_FEATS], ct_train[CT_TARGET]);

t_model.fit(t_train[T_FEATS], t_train[T_TARGET]);

### **CHECK PERFORMANCE**

In [12]:
# y_ct_test = ct_model.predict(ct_test[CT_FEATS])
# y_ct_train = ct_model.predict(ct_train[CT_FEATS])

y_t_test = t_model.predict(t_test[T_FEATS])
y_t_train = t_model.predict(t_train[T_FEATS])

In [13]:
# Mean_squared_error

# print(f"CT test error: {mean_squared_error(y_pred=y_ct_test, y_true=ct_test[CT_TARGET], squared=False)}")
# print(f"CT train error: {mean_squared_error(y_pred=y_ct_train, y_true=ct_train[CT_TARGET], squared=False)}")
print()
print(f"T test error: {mean_squared_error(y_pred=y_t_test, y_true=t_test[T_TARGET], squared=False)}")
print(f"T train error: {mean_squared_error(y_pred=y_t_train, y_true=t_train[T_TARGET], squared=False)}")


T test error: 7269.9180650791395
T train error: 7234.219495584384


In [14]:
# r2_score

# print(f"CT test error: {r2_score(y_pred=y_ct_test, y_true=ct_test[CT_TARGET])}")
# print(f"CT train error: {r2_score(y_pred=y_ct_train, y_true=ct_train[CT_TARGET])}")
print()
print(f"T test error: {r2_score(y_pred=y_t_test, y_true=t_test[T_TARGET])}")
print(f"T train error: {r2_score(y_pred=y_t_train, y_true=t_train[T_TARGET])}")


T test error: 0.3634357891263148
T train error: 0.37004033797566804


--------------------------------
### LGBM REGRESSOR
CT test error:_______8444.855 || <font color='red'>0.34713</font>

CT train error:______8392.133 || <font color='red'>0.35541</font>

T test error:________6927.958 || <font color='red'>0.36787</font>

T train error:_______6838.890 || <font color='red'>0.38061</font>

### We get a very unaccurate prediction with a very low r2 score

After so many attempts with new features, parameter tuning, and different regressors, we conclude to try to <font color='green'>**predict a range of team values**</font>, not the exact value